In [9]:
# !pip install recordlinkage unidecode

In [10]:
import recordlinkage
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from ensemble import ActiveLearner
import recordlinkage.datasets as rl_data
import random
import string
from unidecode import unidecode

In [11]:
np.random.seed(11)

In [12]:
data = rl_data.load_febrl4(return_links=True)
df_org, df_dup, df_links = data[0], data[1], data[2]

In [13]:
# create dict of true links
def def_value():
    return "Not Present"
      
links = defaultdict(def_value)

for org, dup in df_links:
  links[org] = dup

In [14]:
# import name datasets to create dataset
# replace unknown frequencies ('..') with the min (100)
url = 'https://raw.githubusercontent.com/jvalhondo/spanish-names-surnames/master/'
female_df = pd.read_csv(url + 'female_names.csv')
male_df = pd.read_csv(url + 'male_names.csv')
surname_df = pd.read_csv(url +'surnames_freq_ge_100.csv' ).replace('..', 100)

In [16]:
# assuming 50/50 male/female split - sample from first and surnames based on frequency
first_f = female_df.sample(n=df_org.shape[0]//2, weights='frequency', 
                           random_state=1)['name'].reset_index(drop=True)
first_m = male_df.sample(n=df_org.shape[0]-first_f.shape[0], 
                         weights='frequency', random_state=1)['name'].reset_index(drop=True)
first_org = first_f.append(first_m).reset_index(drop=True)
sur_f = surname_df.sample(n=df_org.shape[0], weights='frequency_first', 
                          random_state=1)['surname'].reset_index(drop=True)
sur_l = surname_df.sample(n=df_org.shape[0], weights='frequency_second', 
                          random_state=2)['surname'].reset_index(drop=True)
surname_org = sur_f + ' ' + sur_l

In [17]:
def add_noise(word, switch, flip):
  '''function to add noise to names - replace n letters w/ 
  prob switch^n and flip order if 2 names, w/prob flip'''
  while True:
    if np.random.rand() < switch:
        break
    new_letter = random.choice(string.ascii_uppercase)
    word = word.replace(word[np.random.randint(len(word))], new_letter, 1)

  if ' ' in word and np.random.rand() > flip:
    word = word.split(' ')
    word = word[1] +' ' + word[0]
  return word

In [18]:
# add noise to dup
first_dup = first_org.apply(lambda name: add_noise(name, .5, .5))
surname_dup = surname_org.apply(lambda name: add_noise(name, .5, .5))

In [19]:
# add new names to df
df_org = df_org.sort_index()
df_dup = df_dup.sort_index()
df_org['given_name'] = list(first_org)
df_org['surname'] = list(surname_org)
df_dup['given_name'] = list(first_dup)
df_dup['surname'] = list(surname_dup)

In [20]:
# cleaning
#df_org['given_name'] = df_org['given_name'].apply(lambda name: unidecode(name).lower())
#df_org['surname'] = df_org['surname'].apply(lambda name: unidecode(name).lower())
#df_dup['given_name'] = df_dup['given_name'].apply(lambda name: unidecode(name).lower())
#df_dup['surname'] = df_dup['surname'].apply(lambda name: unidecode(name).lower())

In [21]:
# drop ssn and dob to make linkage more challenging
df_org = df_org.drop(['soc_sec_id', 'date_of_birth'], axis=1)
df_dup = df_dup.drop(['soc_sec_id', 'date_of_birth'], axis=1)

In [23]:
# create indices to compare and set blocking  variable
indexer = recordlinkage.Index()
indexer.block(left_on='postcode', right_on='postcode')
candidate_pairs = indexer.index(df_org, df_dup)

In [24]:
# initialise class
comp = recordlinkage.Compare()

# initialise similarity measurement algorithms
comp.string('given_name', 'given_name', method='jarowinkler')
comp.string('surname', 'surname', method='jarowinkler')
comp.exact('street_number', 'street_number')
comp.string('address_1', 'address_1', method='jarowinkler')
comp.string('address_2', 'address_2', method='jarowinkler')
comp.string('suburb', 'suburb', method='jarowinkler')
# comp.string('postcode', 'postcode', method='jarowinkler')
comp.string('state', 'state', method='jarowinkler')
# comp.string('date_of_birth', 'date_of_birth', method='jarowinkler')
# comp.string('soc_sec_id', 'soc_sec_id', method='jarowinkler')

# the method .compute() returns the DataFrame with the feature vectors
df_compare = comp.compute(candidate_pairs, df_org, df_dup)

In [25]:
def true_class(org, dup):
  '''returns true if org and dup are true pair
  false otherwise'''
  if links[org] == dup:
    return True
  else:
     return False
     
# add true label column to dataframe
df_compare['true_class'] = [true_class(org, dup) for org, dup in df_compare.index]

In [26]:
# pull out n labeled pairs for training - 
# ensure at least 1 match
n=50
while True:
  X_train = df_compare.sample(n=n)
  y_train = X_train['true_class']
  if np.sum(y_train) > 0:
    break
X_train = X_train.drop('true_class', axis=1)

X_test = df_compare.drop(X_train.index, axis=0)
y_test = X_test['true_class']
X_test = X_test.drop('true_class', axis=1)

# check that rows add up and that X_train index not in X_test
assert X_train.shape[0] + X_test.shape[0] == df_compare.shape[0]
assert X_train.index[np.random.randint(n)] not in X_test.index

In [28]:
# test rule based approach (score are ONLY WITHIN BLOCKING!)
# select best threshold value using fscore from the n labeled pairs from above
# note: accuracy, precision and recall are all measured only considering
# the matches that ended up in the same block
# this is because currently we are just trying to work on the classifier
# investigating blocking strategies is a next step
f1_list = []
for thresh in np.linspace(0,10, 101):
  df_train = X_train.copy()
  df_train['pred_class'] = df_train[[col for col in df_train.columns 
                                             if col != 'true_class']].sum(axis=1).values > thresh

  f1_list.append((f1_score(y_train, df_train['pred_class']), thresh))

thresh_best = sorted(f1_list, key=lambda x: x[0])[-1][1]

df_test = X_test.copy()
df_test['pred_class'] = df_test[[col for col in df_test.columns 
                                            if col != 'true_class']].sum(axis=1).values > thresh_best
print('thresh: ', thresh_best)
print('accuracy: ', accuracy_score(y_test, df_test['pred_class']))
print('recall: ',recall_score(y_test, df_test['pred_class']))
print('precision: ', precision_score(y_test, df_test['pred_class']))

thresh:  5.1000000000000005
accuracy:  0.9852235722539304
recall:  0.89978627404417
precision:  1.0


In [29]:
# the classes are relatively separable just from summing the distance measures
df_test = X_test.copy()
df_test['sum'] = df_test[[col for col in df_test.columns 
                                            if col != 'true_class']].sum(axis=1).values
df_test['true_class'] = y_test

fig = px.violin(df_test, y="sum", x="true_class", color="true_class", box=True, points="all")
fig.show()

In [42]:
lr = LogisticRegression(penalty='l2')
AL = ActiveLearner(df_org, df_dup, X_train, y_train, X_test, lr, 1000, 10)

In [52]:
AL.train()

pre train
 training data shape:  (3080, 7)
 training labels shape:  (3080,)
 unlabeled data shape:  (25529, 7)
post train
 training data shape:  (4080, 7)
 training labels shape:  (4080,)
 unlabeled data shape:  (24529, 7)


In [53]:
# note: accuracy, precision and recall are all measured only considering
# the matches that ended up in the same block
# this is because currently we are just trying to work on the classifier
# investigating blocking strategies is a next step
y_pred = AL.model.predict(AL.X_nolabel)
y_true = [true_class(org, dup) for org, dup in AL.X_nolabel.index]
print('accuracy', accuracy_score(y_pred, y_true))
print('recall', recall_score(y_true, y_pred))
print('precision', precision_score(y_true, y_pred))

accuracy 0.9957193526030413
recall 0.9735136658213581
precision 0.9968263127524524


In [51]:
AL.clerical_review()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state
rec-2237-org,DANIELLA,ZELEDON AISSA,15,rylah crescent,brbe is retmt vlg,waratah west,7310,tas
rec-2237-dup-0,DANXEVEA,AISSA ZELEOON,NaN,rylah crescent,NaN,waratah west,7310,tas


Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state
rec-1745-org,MANE,ALMAGRO MADEIRA,23,delprat circuit,erina gardns,maryborough,2430,sa
rec-1745-dup-0,MHNE,MADBIRA ALMAGRO,14,delprat icrcuit,erina gardns,maryborough,2430,NaN


Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state
rec-2880-org,MARIOLA,CORONADO JAREN,7,gruner street,platina,seaforth,5157,vic
rec-4539-dup-0,ANDRES MAURICIO,XRRIOLS MALO,7,eucumbeneq drive,kooyonp,lalor,5157,vic


...

Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state
rec-4796-org,MANUEL MARCELO,BERGADO ARIÑO,51,perry drive,beulah estate,NaN,3043,act
rec-4796-dup-0,YXNUEL MASCELO,BERGADO ARIÑO,15,perry drive,beulah ettate,NaN,3043,act


Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state
rec-3024-org,ILONA,BAÑERES TEN,1,hoseason street,robley house,jamestown,3782,nsw
rec-3024-dup-0,ILONA,TEN BAÑERES,2,NaN,robley house,jamdstown,3782,nsw


Enter label (True/False) :True
all data labeled


In [57]:
# try matching on just given_name and surname, blocking stays the same
df_org2 = df_org[['given_name', 'surname', 'postcode']]
df_dup2 = df_dup[['given_name', 'surname', 'postcode']]

In [59]:
# create indices to compare and set blocking  variable
indexer = recordlinkage.Index()
indexer.block(left_on='postcode', right_on='postcode')
candidate_pairs = indexer.index(df_org2, df_dup2)

In [63]:
# initialise class
comp = recordlinkage.Compare()

# initialise similarity measurement algorithms
comp.string('given_name', 'given_name', method='jarowinkler')
comp.string('surname', 'surname', method='jarowinkler')

# the method .compute() returns the DataFrame with the feature vectors
df_compare2 = comp.compute(candidate_pairs, df_org2, df_dup2)

In [64]:
# label with true class
df_compare2['true_class'] = [true_class(org, dup) for org, dup in df_compare2.index]

In [65]:
# pull out n labeled pairs for training - 
# ensure at least 1 match
n=50
while True:
  X_train = df_compare2.sample(n=n)
  y_train = X_train['true_class']
  if np.sum(y_train) > 0:
    break
X_train = X_train.drop('true_class', axis=1)

X_test = df_compare2.drop(X_train.index, axis=0)
y_test = X_test['true_class']
X_test = X_test.drop('true_class', axis=1)

# check that rows add up and that X_train index not in X_test
assert X_train.shape[0] + X_test.shape[0] == df_compare2.shape[0]
assert X_train.index[np.random.randint(n)] not in X_test.index

In [86]:
# test rule based approach (score are ONLY WITHIN BLOCKING!)
# select best threshold value using fscore from the n labeled pairs from above
# note: accuracy, precision and recall are all measured only considering
# the matches that ended up in the same block
# this is because currently we are just trying to work on the classifier
# investigating blocking strategies is a next step
f1_list = []
for thresh in np.linspace(0,3, 101):
  df_train = X_train.copy()
  df_train['pred_class'] = df_train[[col for col in df_train.columns 
                                             if col != 'true_class']].sum(axis=1).values > thresh

  f1_list.append((f1_score(y_train, df_train['pred_class']), thresh))

thresh_best = sorted(f1_list, key=lambda x: x[0])[-1][1]

df_test = X_test.copy()
df_test['pred_class'] = df_test[[col for col in df_test.columns 
                                            if col != 'true_class']].sum(axis=1).values > thresh_best
print('thresh: ', thresh_best)
print('accuracy: ', accuracy_score(y_test, df_test['pred_class']))
print('recall: ',recall_score(y_test, df_test['pred_class']))
print('precision: ', precision_score(y_test, df_test['pred_class']))

thresh:  1.26
accuracy:  0.9626037326236913
recall:  0.9162117256112035
precision:  0.8437158469945355


In [70]:
lr = LogisticRegression(penalty='l2')
AL = ActiveLearner(df_org2, df_dup2, X_train, y_train, X_test, lr, 1000, 10)

In [84]:
AL.train()

pre train
 training data shape:  (3090, 2)
 training labels shape:  (3090,)
 unlabeled data shape:  (25519, 2)
post train
 training data shape:  (4090, 2)
 training labels shape:  (4090,)
 unlabeled data shape:  (24519, 2)


In [85]:
# note: accuracy, precision and recall are all measured only considering
# the matches that ended up in the same block
# this is because currently we are just trying to work on the classifier
# investigating blocking strategies is a next step
y_pred = AL.model.predict(AL.X_nolabel)
y_true = [true_class(org, dup) for org, dup in AL.X_nolabel.index]
print('accuracy', accuracy_score(y_pred, y_true))
print('recall', recall_score(y_true, y_pred))
print('precision', precision_score(y_true, y_pred))

accuracy 0.9628859252008647
recall 0.7641150922909881
precision 0.9856442577030813


In [83]:
AL.clerical_review()

,given_name,surname,postcode
rec-241-org,GUAYARMINA,MARTOS LLUCH,4211
rec-241-dup-0,GUAYARRINA,LLUCH MARTOS,4211


Enter label (True/False) :True


,given_name,surname,postcode
rec-4834-org,IVO,BAGUE ESPERANZA,2228
rec-4834-dup-0,INZ,BAGUL ESPERANZA,2228


Enter label (True/False) :True


,given_name,surname,postcode
rec-1053-org,ANA DELIA,MONTSERRAT SUAREZ,6210
rec-408-dup-0,ANDREI CATALIN,AQTOLINOS YAGUE,6210


...

Enter label (True/False) :True


,given_name,surname,postcode
rec-4878-org,TORCUATO,ALLOZA DOBLADO,2770
rec-4878-dup-0,BJSCUATO,DOBLADO DLLBZA,2770


Enter label (True/False) :True


,given_name,surname,postcode
rec-4830-org,EDWIN FABRICIO,BRESO PIERAS,2749
rec-4830-dup-0,FABRICIO EDWIN,YUEMODPIERAS,2749


Enter label (True/False) :True
all data labeled
